In [11]:
from autoMMM.bayes import VARModel
import pymc as pm

# Simulate Data 

In [2]:
import numpy as np
import pandas as pd

np.random.seed(123)

# Simulate some endogenous variables with 3 lags
n = 104
y1 = np.random.normal(size=n)
y2 = np.random.normal(size=n)
y3 = np.random.normal(size=n)
for i in range(3, n):
    y1[i] = -0.3 * y1[i - 1] - 0.6 * y1[i - 2] + 0.2 * y2[i - 2] + np.random.normal()
    y2[i] = -0.5 * y2[i - 1] + 0.2 * y3[i - 1] + np.random.normal()
    y3[i] = 0.7 * y3[i - 1] + 0.4 * y3[i - 2] - 0.3 * y3[i - 3] + np.random.normal()

# Simulate some exogenous variable
exog1 = np.random.normal(size=n)

# Put everything in pandas DataFrames
endog_df = pd.DataFrame({"x": y1, "Awareness": y2, "Long-Term Base": y3})
endog_df = endog_df.set_index(pd.date_range(start="2022-01-01", periods=n, freq="D"))

exog_df = pd.DataFrame({"Media": exog1})
exog_df = exog_df.set_index(pd.date_range(start="2022-01-01", periods=n, freq="D"))





# Specify the VAR parameters & priors

In [ ]:
# Test VARModel
n_lags = 3
n_eqs = 3
priors = {
    "lag_coefs": {"mu": 0, "sigma": 0.1},
    "alpha": {"mu": 0, "sigma": 0.1},
    "coefs": {"mu": 0, "sigma": 0.1},
    "noise_chol": {"eta": 2, "sigma": 0.1},
    "noise": {"sigma": 1},
}



# Build the model & fit

In [3]:
model = VARModel(n_lags, n_eqs, endog_df, priors, exog_df=exog_df, mv_norm=False, prior_checks=False)
idata = model.fit(draws=2000, tune=500, target_accept=0.9)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lag_coefs, alpha, exog_coefs, noise]


Sampling 4 chains for 500 tune and 2_000 draw iterations (2_000 + 8_000 draws total) took 42 seconds.
Sampling: [obs]


# Compute Impulse Response Functions

In [8]:
# Compute and plot impulse response function
irf_df = model.impulse_response_function(num_periods=10, shock_var=1, shock_magnitude=1, cumsum=True, plot=False)

In [9]:
irf_df

,x,Awareness,Long-Term Base
0,0.000000,1.000000,0.000000
1,-0.008682,0.908152,0.052692
2,-0.006731,0.919713,0.059225
3,-0.007045,0.918940,0.061334
4,-0.006996,0.919135,0.061750
5,-0.007003,0.919138,0.061853
6,-0.007002,0.919144,0.061876
7,-0.007002,0.919144,0.061881
8,-0.007002,0.919144,0.061882
9,-0.007002,0.919144,0.061883


# Inspect model coefficients

In [13]:
pm.summary(idata, var_names=["lag_coefs"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"lag_coefs[x, 1, x]",-0.179,0.070,-0.318,-0.053,0.001,0.000,13500.0,6499.0,1.0
"lag_coefs[x, 1, Awareness]",0.004,0.064,-0.116,0.123,0.001,0.001,15658.0,6469.0,1.0
"lag_coefs[x, 1, Long-Term Base]",-0.072,0.063,-0.186,0.049,0.001,0.000,14016.0,6388.0,1.0
"lag_coefs[x, 2, x]",-0.274,0.069,-0.398,-0.140,0.001,0.000,13926.0,6435.0,1.0
"lag_coefs[x, 2, Awareness]",0.050,0.069,-0.080,0.177,0.001,0.001,12485.0,6365.0,1.0
"lag_coefs[x, 2, Long-Term Base]",0.071,0.068,-0.059,0.199,0.001,0.001,14226.0,6136.0,1.0
"lag_coefs[x, 3, x]",0.100,0.068,-0.026,0.227,0.001,0.000,15721.0,6654.0,1.0
"lag_coefs[x, 3, Awareness]",-0.080,0.068,-0.206,0.046,0.001,0.001,14539.0,6038.0,1.0
"lag_coefs[x, 3, Long-Term Base]",0.008,0.062,-0.103,0.131,0.001,0.001,14979.0,6196.0,1.0
"lag_coefs[Awareness, 1, x]",-0.047,0.074,-0.182,0.097,0.001,0.001,12987.0,6154.0,1.0
